<a href="https://colab.research.google.com/github/present42/PyTorchPractice/blob/main/Fluent_Python_ch5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 5 Data Class Builders
 - `collections.namedtuple`
 - `typing.NamedTuple`
 - `@dataclasses.dataclass`

Note. `typing.TypedDict` does not build concrete classes that you can instantiate.

In [1]:
class Coordinate:
  def __init__(self, lat, lon):
    self.lat = lat
    self.lon = lon

In [2]:
moscow = Coordinate(55.76, 37.62)
moscow

In [3]:
location = Coordinate(55.76, 37.62)
location == moscow

False

The data class builders provide the necessary `__init__`, `__repr__` and `__eq__` methods automatically.

In [4]:
from collections import namedtuple

Coordinate = namedtuple('Coordinate', 'lat lon')
issubclass(Coordinate, tuple)

True

In [5]:
moscow = Coordinate(55.756, 37.617)
moscow # Useful __repr__

Coordinate(lat=55.756, lon=37.617)

In [6]:
moscow == Coordinate(lat=55.756, lon=37.617) # meaningful __eq__

True

Newer NamedTuple provides the same functionality, adding a type annotation to each field

In [7]:
import typing

Coordinate = typing.NamedTuple('Coordinate', [('lat', float), ('lon', float)])

In [8]:
issubclass(Coordinate, tuple)

True

In [9]:
typing.get_type_hints(Coordinate)

{'lat': float, 'lon': float}

In [10]:
Coordinate = typing.NamedTuple('Coordinate', lat=float, lon=float)

**Warning**
Although `NamedTuple` appears in the `class` statement as a superclass, it's actually not. It uses the advanced functionality of a metaclass to customize the creation of the user's class.

In [12]:
from typing import NamedTuple

class Coordinate(NamedTuple):
  lat: float
  lon: float

  def __str__(self):
    ns = 'N' if self.lat >= 0 else 'S'
    we = 'E' if self.lon >= 0 else 'W'
    return f'{abs(self.lat):1.f}°{ns}, {abs(self.lon):1.f}°{we}'

In [17]:
issubclass(Coordinate, typing.NamedTuple)

TypeError: issubclass() arg 2 must be a class, a tuple of classes, or a union

In [14]:
issubclass(Coordinate, tuple)

True

In [25]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Coordinate:
  lat: float
  lon: float = 10

  def __str__(self):
    ns = 'N' if self.lat >= 0 else 'S'
    we = 'E' if self.lon >= 0 else 'W'
    return f'{abs(self.lat):1.f}°{ns}, {abs(self.lon):1.f}°{we}'

The difference is in the `class` statement itslef. The `@dataclass` decorator does not depend on inheritance or a metaclass.

### Mutable instances

By default, `@dataclass` produces mutable classes. But the decorator accepts a keyword argument `frozen`. When `frozen=True`, the class will raise an exception if you try to assign a value to a field after the instance is initialized.

### Class statement syntax
Only `typing.NamedTuple` and `dataclass` support the regular `class` statement syntax.

### Construct dict

In [26]:
import dataclasses

dataclasses.asdict(Coordinate(37.28, 127.0))

{'lat': 37.28, 'lon': 127.0}

### Get field names and default values

In [29]:
[f.default for f in dataclasses.fields(Coordinate(37.28, 127.0))]

[<dataclasses._MISSING_TYPE at 0x7fdfb9293dc0>, 10]

In [31]:
typing.get_type_hints(Coordinate)

{'lat': float, 'lon': float}

In [34]:
# namedtuple._replace returns a new instance with some attribute values repalced
moscow._replace(lon=38)

Coordinate(lat=55.756, lon=37.617)

In [38]:
suwon = Coordinate(37.28, 127.0)
dataclasses.replace(suwon, lat=37.29)

Coordinate(lat=37.29, lon=127.0)

## Classic Named Tuples

In [39]:
from collections import namedtuple
# arguments: classname and iterables of string (or a single space-delimited string)
City = namedtuple('City', 'name country population coordinates')
tokyo = City('Tokyo', 'JP', '36.933', (35.689722, 139.691667))
tokyo

City(name='Tokyo', country='JP', population='36.933', coordinates=(35.689722, 139.691667))

In [40]:
tokyo.population

'36.933'

In [41]:
tokyo[1]

'JP'

A named tuple offers a few attributes and methods in addition to those inherited from tuples.

Ex. `_fields` class attribute, `_make(iterable)` class method, `_asdict()` instance method

In [42]:
City._fields

('name', 'country', 'population', 'coordinates')

In [43]:
Coordinate = namedtuple('Coordinate', 'lat lon')

In [44]:
delhi_data = ('Delhi NCR', 'IN', 21.935, Coordinate(28.613889, 77.208889))

In [45]:
delhi = City._make(delhi_data) # _make buildes City from an iterable

In [46]:
delhi._asdict()

{'name': 'Delhi NCR',
 'country': 'IN',
 'population': 21.935,
 'coordinates': Coordinate(lat=28.613889, lon=77.208889)}

`._asdict()` is useful to serialize the data in JSON format

In [47]:
import json

json.dumps(delhi._asdict())

'{"name": "Delhi NCR", "country": "IN", "population": 21.935, "coordinates": [28.613889, 77.208889]}'

Since Python 3.7, `namedtuple` accepts the `defaults` keyword-only argument providing an iterable of N default values for each of the N rightmost fields of the class.

In [48]:
Coordinate = namedtuple('Coordinate', 'lat lon reference', defaults=['WGS84'])
Coordinate(0, 0)

Coordinate(lat=0, lon=0, reference='WGS84')

In [49]:
Coordinate._field_defaults

{'reference': 'WGS84'}

#### Hacking a namedtuple to inject a method

In [50]:
Card = namedtuple('Card', ['rank', 'suit'])

In [51]:
class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

In [55]:
Card.suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)
def spades_high(card):
  rank_value = FrenchDeck.ranks.index(card.rank)
  suit_value = card.suit_values[card.suit]
  return rank_value * len(card.suit_values) + suit_value

Card.overall_rank = spades_high

In [53]:
lowest_card = Card('2', 'clubs')
highest_card = Card('A', 'spades')

In [56]:
lowest_card.overall_rank()

0

In [57]:
highest_card.overall_rank()

51